In [3]:
import urllib
import scipy
from bs4 import *
import requests
from bs4 import BeautifulSoup
import os
import lxml
import html5lib
import numpy as np
import simplejson

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
from selenium import webdriver

In [39]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# **Get links for the speakers from Librivox**


In [ ]:
ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()

In [45]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
path = r'C:\Users\sonal\.wdm\drivers\chromedriver\win32\98.0.4758.80\chromedriver.exe'
browser = webdriver.Chrome(executable_path=path,options=options)

C:\Users\sonal\AppData\Local\Temp\ipykernel_2352\1684493885.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=path,options=options)


In [46]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
book_links = []

##browser = webdriver.PhantomJS(path)
#browser = webdriver.Chrome('chr,options=options)
#driver = webdriver.Chrome(service=service_object)

for i in range(200):
    print("i=",i)
    url = ("https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=&sort_order=catalog_date&search_page={}&search_form=advanced").format(i)
    url = ("https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=&sort_order=catalog_date&search_page={}&search_form=advanced").format(i)
    print(url)
    
    browser.get(url)
    element = WebDriverWait(browser, 100).until(
    EC.presence_of_element_located((By.CLASS_NAME , "catalog-result")))
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    ul_tag = soup.find('ul', {'class': 'browse-list'})
    
    for li_tag in ul_tag.find_all('li', {'class': 'catalog-result'}):
        result_data = li_tag.find('div', {'class': 'result-data'})
        book_meta = result_data.find('p', {'class': 'book-meta'})
        link = result_data.a["href"]
        if str(book_meta).find("Complete") and link not in book_links:
            book_links.append(link)
            
    print(len(book_links))
    
browser.quit()

i= 0
https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=&sort_order=catalog_date&search_page=0&search_form=advanced
25
i= 1
https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=&sort_order=catalog_date&search_page=1&search_form=advanced
25
i= 2
https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=&sort_order=catalog_date&search_page=2&search_form=advanced
38
i= 3
https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=&sort_order=catalog_date&search_page=3&search_form=advanced
60
i= 4
https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=&sort_order=catalog_date&search_page=4&search_form=advanced
78
i= 5
https://librivox.org/search?title=&author=&reader=&keywords=

In [47]:
f = open('book_links.txt', 'w')
simplejson.dump(book_links, f)
f.close()

# **Search for speakers list**

In [ ]:
##browser = webdriver.PhantomJS()
browser = webdriver.Chrome(executable_path=path,options=options)
reader_list = []
download_links = []
download_sizes = []

repreader = 0

for i in range(len(book_links)):
    print("i=",i)
    link = book_links[i]
    
    browser.get(link)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    product_details = soup.find('dl', {'class': 'product-details clearfix'})
    
    if product_details == None:
        continue
    
    product_details_list = product_details.find_all("dd")
    
    reader = product_details_list[3].get_text()
    size_mb = product_details_list[1].get_text()
    
    try:
        size_mb = float(size_mb.replace('MB',''))
    except:
        continue
    
    if reader not in reader_list:
        
        reader_list.append(reader)
        download_sizes.append(size_mb)

        listen_download = soup.find('dl', {'class': 'listen-download clearfix'})
        zip_download = listen_download.a["href"]

        print(reader, str(len(reader_list)) + "/" + str(i+1) + " potentials")

        download_links.append(zip_download)
            
    else:
        repreader += 1
        print("repeat reader " + str(repreader))
            
        
browser.quit()

In [3]:
import simplejson
f = open('download_links.txt', 'w')
simplejson.dump(download_links, f)
f.close()

NameError: name 'download_links' is not defined

In [ ]:
f = open('download_sizes.txt', 'w')
simplejson.dump(download_sizes, f)
f.close()

In [ ]:
print(download_links)

In [ ]:
from requests.exceptions import InvalidURL

# **Download audiobooks of different sizes**

In [ ]:

import os, os.path
def count_files():
    dir = "/content/drive/My Drive/btech project/voice-classification-master/audio"
    list = [file for file in os.listdir(dir) if file.endswith(".zip")] # dir is your directory path
    number_files = len(list)
    return number_files

counter = 193

for link, size in zip(download_links, download_sizes):

    if size >= 50 and size <= 100:
        localDestination = "/content/drive/My Drive/btech project/voice-classification-master/audio/audio{}.zip".format(counter)
        try:
          resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
        except urllib.error.HTTPError as exception:
          print(exception)
        except InvalidURL as err:
          print(err)
        except urllib.error.URLError as exception:
          print(exception)

        counter += 1  


cnt2 =  0
num = count_files()

if num < 200:
    for link, size in zip(download_links, download_sizes):
        if size > 100 and size <= 150:
            localDestination = "/content/drive/My Drive/btech project/voice-classification-master/audio/audio{}.zip".format(counter)
            try:
              resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
            except urllib.error.HTTPError as exception:
              print(exception)
            except InvalidURL as err:
              print(err)
            counter += 1
        cnt2 += 1

num = count_files()

if num < 200:
    for link, size in zip(download_links, download_sizes):
        if size > 150 and size <= 200:
            localDestination = dir + "/content/drive/My Drive/btech project/voice-classification-master/audio/audio{}.zip".format(counter)
        try:
          resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
        except urllib.error.HTTPError as exception:
          print(exception)
        except InvalidURL as err:
          print(err)
        counter += 1
        
num = count_files()

if num < 200:
    for link, size in zip(download_links, download_sizes):
        if size > 200 and size <= 250:
            localDestination = dir + "/content/drive/My Drive/btech project/voice-classification-master/audio/audio{}.zip".format(counter)
        try:
          resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
        except urllib.error.HTTPError as exception:
          print(exception)
        except InvalidURL as err:
          print(err)
        counter += 1
        
num = count_files()

if num < 200:
    for link, size in zip(download_links, download_sizes):
        if size > 250 and size <= 300:
            localDestination = dir + "/content/drive/My Drive/btech project/voice-classification-master/audio/audio{}.zip".format(counter)
        try:
          resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
        except urllib.error.HTTPError as exception:
          print(exception)
        except InvalidURL as err:
          print(err)
        counter += 1
            
num = count_files()

if num < 200:
    for link, size in zip(download_links, download_sizes):
        if size > 300 and size <= 350:
            localDestination = dir + "/content/drive/My Drive/btech project/voice-classification-master/audio/audio{}.zip".format(counter)
        try:
          resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
        except urllib.error.HTTPError as exception:
          print(exception)
        except InvalidURL as err:
          print(err)
        counter += 1
            
num = count_files()

if num < 200:
    for link, size in zip(download_links, download_sizes):
        if size > 350 and size <= 400:
            localDestination = dir + "/content/drive/My Drive/btech project/voice-classification-master/audio/audio{}.zip".format(counter)
        try:
          resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
        except urllib.error.HTTPError as exception:
          print(exception)
        except InvalidURL as err:
          print(err)
        counter += 1
    
        


KeyboardInterrupt: ignored

# **Remove silences more than 5 seconds**

In [ ]:
import os
import subprocess

import time

start_time = time.time()

DIRECTORY = '/content/drive/My Drive/btech project/voice-classification-master/audio/'

speaker_nums = [file.strip('audio') for file in os.listdir(DIRECTORY) if file.startswith("audio")]

speaker_paths = [os.path.join(DIRECTORY, path) for path in os.listdir(DIRECTORY) if path.startswith("audio")]


for path, num in zip(speaker_paths, speaker_nums):
    
    print(path, num)
    
    os.system(("mp3wrap {a}/combined{b}.mp3 {a}/*.mp3;"
               "ffmpeg -i {a}/combined{b}_MP3WRAP.mp3 -ss 0 -t 2700 -acodec copy {a}/combined{b}_45m.mp3;"
               
               "mkdir {a}/combined_45m;"
               "mv {a}/combined{b}_45m.mp3 {a}/combined_45m;"
               
               "mkdir {a}/combined_MP3;"
               "mv {a}/combined{b}_MP3WRAP.mp3 {a}/combined_MP3;"
               
               "rm {a}/*.mp3;"
               
               "ffmpeg -i {a}/combined_45m/combined{b}_45m.mp3 -acodec pcm_s16le -ac 1 -ar 16000 {a}/combined_45m/combined{b}_45m.wav;"
               "rm {a}/combined_45m/combined{b}_45m.mp3;"
               "sox {a}/combined_45m/combined{b}_45m.wav {a}/combined_45m/combined{b}_45m_fixed.wav silence 1 0.1 1% -1 0.5 1%;"
               "rm {a}/combined_45m/combined{b}_45m.wav;"
               
               "mkdir {a}/wave_seg_5sec_45m;"
               "ffmpeg -i {a}/combined_45m/combined{b}_45m_fixed.wav -f segment -segment_time 5 -c copy {a}/wave_seg_5sec_45m/out%03d.wav;"
               
               "mkdir {a}/specs_5sec_45m;"
            
              ).format(a=path, b=num))
    
    wav_seg_path = "{a}/wave_seg_5sec_45m".format(a=path, b=num)
    wav_segs = [os.path.join(wav_seg_path, wav_seg) for wav_seg in os.listdir(wav_seg_path) if wav_seg.startswith("out")]
    
    cnt=0
    for wav_seg in wav_segs:
        os.system("sox {wav} -n spectrogram -r -o {a}/specs_5sec_45m/out{c}.png".format(a=path, b=num, wav=wav_seg, c=cnt))
        cnt+=1
        
    os.system("rm -r {a}/combined_45m".format(a=path))
    
print("My program took", time.time() - start_time, "to run")

In [ ]:
DIR = "../../Audio"

speaker_list = [os.path.join(DIR, speaker) for speaker in os.listdir(DIR) if speaker.startswith("audio")]

new_dir = "../../spect_files"
os.system("mkdir " + new_dir)

for speaker in speaker_list:
    
    specs_folder = speaker + "/specs_5sec_45m"
    new_subdir = new_dir + "/" + speaker.strip(DIR)

    print(speaker.strip(DIR))
    
    os.system("mkdir " + new_subdir)
    os.system("mv " + specs_folder + " " + new_subdir)